In [1]:
# --- Instalaciones y Configuraciones Iniciales ---
# Necesitamos instalar Gradio, pycocotools y la Object Detection API
!pip install gradio
!pip install pycocotools # Necesario para la evaluación COCO si no está ya
!pip install tf-models-official==2.5.1 # Asegura que tengamos las utilidades de Object Detection

In [2]:
# --- Instalaciones y Configuraciones Iniciales ---
# Necesitamos instalar Gradio, pycocotools y la Object Detection API
# Primero desinstalamos versiones conflictivas de protobuf
!pip uninstall protobuf -y
!pip uninstall libprotobuf -y # Algunas instalaciones pueden tener este también

# Instalamos una versión compatible de protobuf
# La versión 3.20.x es una opción segura. 3.20.3 es la última de esa serie.
# El error indica que con protobuf >= 3.19.0 y <= 3.20.x debería funcionar si el código generado es reciente.
# Ya que el código generado es el problema, vamos a asegurarnos de que la versión de protobuf
# es compatible con el protoc que vamos a usar o que el protoc genera código compatible.
# Mantengamos 3.20.3 como base.
!pip install protobuf==3.20.3

# Reinstalamos pycocotools y tf-models-official por si las dudas (dependen de protobuf)
!pip install pycocotools
!pip install tf-models-official

# --- Clonar el repositorio de TensorFlow Models y configurar PYTHONPATH ---
# Esto sigue siendo necesario para las utilidades de Object Detection
import os
import sys

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.65.5 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.65.5 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.


In [3]:
# Directorio donde queremos clonar los modelos
MODELS_RESEARCH_DIR = '/content/models/research'

if not os.path.exists(MODELS_RESEARCH_DIR):
    print("Clonando TensorFlow Models...")
    !git clone https://github.com/tensorflow/models.git /content/models # Clonar directamente a /content/models
    print("Modelos clonados.")

    # Compilar protos después de clonar y asegurarnos de estar en el directorio correcto
    print("Compilando archivos proto...")
    %cd {MODELS_RESEARCH_DIR}

    # Ejecutar protoc. Nos aseguramos de que estamos en el directorio correcto.
    # Si 'protoc' no está en el PATH o no es la versión correcta, esto podría fallar.
    # En un entorno Colab, el 'protoc' del sistema suele ser compatible con 3.x después de instalar protobuf 3.x.
    %cd /content/

    print("Archivos proto compilados.")
else:
    print("El repositorio de TensorFlow Models ya existe.")
    # Si el repositorio ya existe, aún debemos asegurarnos de que los protos estén compilados.
    # Es buena práctica recompilarlos por si acaso o si el entorno cambió.
    print("Re-compilando archivos proto...")
    %cd {MODELS_RESEARCH_DIR}
    !protoc object_detection/protos/*.proto --python_out=.
    %cd /content/ # Volver a la raíz /content/
    print("Archivos proto re-compilados.")

%cd {MODELS_RESEARCH_DIR}
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .
%cd /content/

# Agregar models/research y models/research/slim al PYTHONPATH
if '/content/models/research' not in sys.path:
    sys.path.append('/content/models/research')
if '/content/models/research/slim' not in sys.path:
    sys.path.append('/content/models/research/slim')

print("Configuración de protobuf, entorno y PYTHONPATH completada.")

# --- Configuración de entorno para Protobuf (ya lo tenías, lo mantenemos) ---
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

El repositorio de TensorFlow Models ya existe.
Re-compilando archivos proto...
/content/models/research
[Errno 2] No such file or directory: '/content/ # Volver a la raíz /content/'
/content/models/research
Archivos proto re-compilados.
/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697325 sha256=f81fbec08e69d4f9ac1f0032698e86b67303dbe3b34bf5f65ca0f5a4b4e0ee67
  Stored in directory: /tmp/pip-ephem-wheel-cache-aa2lwhe5/wheels/95/4a/63/b2d36ca06eab841de19a38993ffaf2beac152a44539bc642a6
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1
/content
Configuración de protobuf, entorno y PYTHONPATH completada.
env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [19]:
# --- Importaciones ---
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as tf_ops
import numpy as np
from PIL import Image
import gradio as gr
import io

print("Importaciones iniciales completadas.")

# --- Configuración de Rutas de tu Modelo en Google Drive ---
# MONTAR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive')
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as tf_ops
import numpy as np
from PIL import Image
import gradio as gr
import io

print("Importaciones y configuraciones iniciales completadas.")

Importaciones iniciales completadas.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Importaciones y configuraciones iniciales completadas.


In [20]:
# Rutas donde subiste tus archivos en Google Drive
# AJUSTA ESTAS RUTAS A DONDE PUSISTE TUS ARCHIVOS EN TU GOOGLE DRIVE
PATH_TO_GDRIVE_MODEL_ZIP = '/content/gdrive/MyDrive/tp_integrador/od_tf_api/exported_model.zip'
PATH_TO_GDRIVE_LABEL_MAP = '/content/gdrive/MyDrive/tp_integrador/od_tf_api/label_map.pbtxt'

# Directorio de destino para descomprimir el modelo en Colab
COLAB_MODEL_DIR = '/content/exported_model'
COLAB_LABEL_MAP_PATH = '/content/label_map.pbtxt'
PATH_TO_SAVED_MODEL = os.path.join(COLAB_MODEL_DIR, 'saved_model')

# --- Descompresión del Modelo y Copia del Label Map ---
print(f"Descomprimiendo modelo desde {PATH_TO_GDRIVE_MODEL_ZIP} a {COLAB_MODEL_DIR}...")
!unzip -q {PATH_TO_GDRIVE_MODEL_ZIP} -d '/content/' # -q para modo silencioso
print("Modelo descomprimido.")

print(f"Copiando label_map desde {PATH_TO_GDRIVE_LABEL_MAP} a {COLAB_LABEL_MAP_PATH}...")
!cp {PATH_TO_GDRIVE_LABEL_MAP} {COLAB_LABEL_MAP_PATH}
print("label_map copiado.")

Descomprimiendo modelo desde /content/gdrive/MyDrive/tp_integrador/od_tf_api/exported_model.zip a /content/exported_model...
Modelo descomprimido.
Copiando label_map desde /content/gdrive/MyDrive/tp_integrador/od_tf_api/label_map.pbtxt a /content/label_map.pbtxt...
label_map copiado.


In [21]:
# --- Cargar el Modelo y el Mapa de Etiquetas ---
try:
    print(f"Cargando modelo exportado desde {PATH_TO_SAVED_MODEL}...")
    detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    print("Asegúrate de que la ruta PATH_TO_SAVED_MODEL sea correcta y que el modelo esté bien exportado.")
    sys.exit(1) # Detiene la ejecución si el modelo no carga

try:
    print(f"Cargando mapa de etiquetas desde {COLAB_LABEL_MAP_PATH}...")
    category_index = label_map_util.create_category_index_from_labelmap(COLAB_LABEL_MAP_PATH, use_display_name=True)
    print("Mapa de etiquetas cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el mapa de etiquetas: {e}")
    print("Asegúrate de que la ruta COLAB_LABEL_MAP_PATH sea correcta y el archivo label_map.pbtxt exista.")
    sys.exit(1) # Detiene la ejecución si el label_map no carga

Cargando modelo exportado desde /content/exported_model/saved_model...
Modelo cargado exitosamente.
Cargando mapa de etiquetas desde /content/label_map.pbtxt...
Mapa de etiquetas cargado exitosamente.


In [22]:
caracteres = "ABCDEFGHIJKLMNOPQRSTUVWXYZÑabcdefghijklmnopqrstuvwxyzñáéíóúü0123456789.,;:?¿!¡-—&*/+=-_@#$%"

# Eliminar duplicados preservando orden
caracteres = ''.join(dict.fromkeys(caracteres))

label_list = [{"id": idx, "label": c} for idx, c in enumerate(caracteres, start=1)]


def get_char_by_id(id, labels=label_list):
    for item in labels:
        if item["id"] == id:
            return item["label"]
    return None

In [24]:
import cv2
# --- Función de Predicción para Gradio ---
def convert_normalized_box_to_pixels(box_normalized, image_width, image_height):
    # Asegurarse de que las coordenadas estén en el orden esperado (ymin, xmin, ymax, xmax)
    ymin_norm, xmin_norm, ymax_norm, xmax_norm = box_normalized

    # Convertir a coordenadas de píxeles
    ymin_pixel = int(ymin_norm * image_height)
    xmin_pixel = int(xmin_norm * image_width)
    ymax_pixel = int(ymax_norm * image_height)
    xmax_pixel = int(xmax_norm * image_width)

    return (xmin_pixel, ymin_pixel, xmax_pixel, ymax_pixel)

def predict_image(input_image_path):
    """
    Realiza la detección de objetos en una imagen y devuelve la imagen con las detecciones.
    """
    if isinstance(input_image_path, str):
      image = Image.open(input_image_path).convert("RGB")
    elif isinstance(input_image_path, (np.ndarray, Image.Image)):
      image = Image.fromarray(input_image_path).convert("RGB")
    else:
      raise TypeError("Tipo de entrada de imagen no soportado.")


    image_np = np.array(image)

    # El modelo espera un batch de imágenes, así que expandimos las dimensiones.
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Realizar la inferencia
    detections = detect_fn(input_tensor)
    #print(format(detections))
    # Procesar las detecciones para visualización
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Las clases se devuelven como floats, convertimos a int64
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Visualizar los resultados en la imagen original
    image_np_with_detecciones = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detecciones,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        # Elimina o comenta esta línea:
        # use_display_name=True,
        max_boxes_to_draw=20,
        min_score_thresh=0.75,
        agnostic_mode=False
    )
    for i in range(detections['num_detections']):
        score = detections['detection_scores'][i]
        box = detections['detection_boxes'][i]
        class_id = detections['detection_classes'][i]
        letra = get_char_by_id(class_id)
        h, w = image_np_with_detecciones.shape[:2]
        if score < 0.5:
            continue
        box=convert_normalized_box_to_pixels(box, w, h)
        cv2.rectangle(image_np_with_detecciones, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 1)
        cv2.putText(image_np_with_detecciones, f"{letra}: score:{score:.2f}", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image_np_with_detecciones


# --- Lanzar la Interfaz de Gradio ---
print("Lanzando interfaz de Gradio...")
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Sube una imagen"),
    outputs=gr.Image(type="numpy", label="Imagen con Detecciones"),
    title="Detector de Caracteres Personalizado",
    description="Sube una imagen y el modelo detectará los caracteres."
    # La línea 'examples' ha sido eliminada.
)

iface.launch(debug=True, share=True)

Lanzando interfaz de Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d92eed3accf97b7813.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d92eed3accf97b7813.gradio.live
